In [1]:
# !sudo apt-get update
# !sudo apt-get install libssl1.0.0 libasound2
# !pip3 install azure-cognitiveservices-speech
import azure.cognitiveservices.speech as speechsdk
#!pip install SpeechRecognition
%cd /content/drive/My Drive/ColabNotebooks/RPL_Language/
import json
from os import path
import os
import time
import wave
import moviepy.editor as mp
from moviepy.editor import *
from moviepy.editor import VideoFileClip
import pandas as pd
import os

#just do once to convert mp4 to wav
directory = '/content/drive/My Drive/ColabNotebooks/RPL_Language/collab-SLA/data_evaluation/'
# for file in os.listdir(directory+'Videos/'):
#      filename = os.fsdecode(file)
#      clip = VideoFileClip(directory+'Videos/'+filename)
#      clip.audio.write_audiofile(directory+'wav_files/'+filename[:-4]+'.wav')

try:
    import azure.cognitiveservices.speech as speechsdk
except ImportError:
    print("""
    Importing the Speech SDK for Python failed.
    Refer to
    https://docs.microsoft.com/azure/cognitive-services/speech-service/quickstart-python for
    installation instructions.
    """)
    import sys
    sys.exit(1)


# Set up the subscription info for the Speech Service:
# Replace with your own subscription key and service region (e.g., "westus").
speech_key, service_region = "cc9e939a93294ac1a93ca3b94e0914ff", "eastus"


def speech_recognize_continuous_from_file(wav_file):
    """performs continuous speech recognition with input from an audio file"""
    # <SpeechContinuousRecognitionWithFile>
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    speech_config.request_word_level_timestamps()
    audio_config = speechsdk.audio.AudioConfig(filename='/content/drive/My Drive/ColabNotebooks/RPL_Language/collab-SLA/data_evaluation/wav_files/'+wav_file)

    # Replace the languages with your languages in BCP-47 format, e.g. fr-FR.
    # Please see https://docs.microsoft.com/azure/cognitive-services/speech-service/language-support
    # for all supported languages

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,language = "sv-SE", audio_config=audio_config)

    done = False

    all_results = []
    jsons = []
    def handle_final_result(evt):
      all_results.append(evt.result.text)
      jsons.append(evt.result.json)
      df_results = pd.DataFrame(all_results)
      #df_results.to_csv('results.csv', index=False)
      df_jsons = pd.DataFrame(jsons)
      df_jsons.to_csv('/content/drive/My Drive/ColabNotebooks/RPL_Language/collab-SLA/data_evaluation/csvs/' + wav_file +'_jsons.csv', index=False)

    def stop_cb(evt):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    # Connect callbacks to the events fired by the speech recognizer
    #speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    #speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    #speech_recognizer.recognized.connect(lambda evt: print('JSON: {}'.format(evt.result.json)))
    speech_recognizer.recognized.connect(handle_final_result)
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()




/content/drive/My Drive/ColabNotebooks/RPL_Language


In [2]:
#directory = '/content/drive/My Drive/ColabNotebooks/RPL_Language/collab-SLA/data_evaluation/wav_files/'
print('saved at '+'/content/drive/My Drive/ColabNotebooks/RPL_Language/collab-SLA/data_evaluation/' +'speech_csvs/' + 'file_name' +'_jsons.csv')
for file in os.listdir('/content/drive/My Drive/ColabNotebooks/RPL_Language/collab-SLA/data_evaluation/wav_files/'):
    wav_file = os.fsdecode(file)
    print(wav_file)
    result = speech_recognize_continuous_from_file(wav_file)

saved at /content/drive/My Drive/ColabNotebooks/RPL_Language/collab-SLA/data_evaluation/speech_csvs/file_name_jsons.csv
10-01-2020-Furhat__100101.wav
SESSION STARTED: SessionEventArgs(session_id=3b180bb95cc4409993c5cb211f01ea70)
CANCELED SpeechRecognitionCanceledEventArgs(session_id=3b180bb95cc4409993c5cb211f01ea70, result=SpeechRecognitionResult(result_id=ac42a33849d0465f82e15d64407f15d6, text="", reason=ResultReason.Canceled))
CLOSING on SpeechRecognitionCanceledEventArgs(session_id=3b180bb95cc4409993c5cb211f01ea70, result=SpeechRecognitionResult(result_id=ac42a33849d0465f82e15d64407f15d6, text="", reason=ResultReason.Canceled))
SESSION STOPPED SessionEventArgs(session_id=3b180bb95cc4409993c5cb211f01ea70)
CLOSING on SessionEventArgs(session_id=3b180bb95cc4409993c5cb211f01ea70)
10-01-2020-Furhat__9697.wav
SESSION STARTED: SessionEventArgs(session_id=983098cae5394ce59a589c1ab2bf1797)
CANCELED SpeechRecognitionCanceledEventArgs(session_id=983098cae5394ce59a589c1ab2bf1797, result=SpeechR

In [16]:
import os
import pandas as pd
import json
%cd /content/drive/My Drive/ColabNotebooks/RPL_Language/
files = []
for filename in os.listdir('/content/drive/My Drive/ColabNotebooks/RPL_Language/collab-SLA/data_evaluation/csvs'):
    src = '/content/drive/My Drive/ColabNotebooks/RPL_Language/collab-SLA/data_evaluation/csvs/'+filename
    files.append(src)

for file in files:
    time = pd.read_csv(file)
    timedf = pd.DataFrame(time)
    timedf.columns = ['string']
    time_new = pd.json_normalize(timedf.string.apply(json.loads))
    time_new = time_new.drop(['RecognitionStatus','Id','NBest'],axis=1)
    time_new['Offset'] = time_new['Offset']/10000000
    time_new['Duration'] = time_new['Duration']/10000000
    print(time_new)
    print(file[84:-14])
    time_new.to_csv('/content/drive/My Drive/ColabNotebooks/RPL_Language/collab-SLA/data_evaluation/speech_csvs/'+file[84:-14]+'_speech.csv')

/content/drive/My Drive/ColabNotebooks/RPL_Language
                                           DisplayText  Duration   Offset
0                                       Fick ju gå in.      1.60     1.97
1                                                 Den.      0.22     8.06
2                                                 Hmm.      1.09    17.11
3    Första året som vi skulle kunna träffas så bli...      6.00    21.25
4                                                 Hmm.      1.50    33.08
..                                                 ...       ...      ...
124  Ja hmm vissa filmer kan också ha en sån men de...     20.24   994.57
125                                          Spelande.      0.90  1016.71
126                                                Du.      0.58  1019.67
127                             20 s ja men precis då.      3.08  1027.14
128                  Ser vi kommit hmm vilken god väg.      2.69  1032.25

[129 rows x 3 columns]
10-01-2020-Furhat__100101
          